In [64]:
import pandas as pd
import json

In [65]:
main_df = pd.read_csv("data/db_main.csv")
other_df = pd.read_csv("data/pizza_indexes.csv")


In [66]:
other_df.sample(5)

,Rank,State,Initials,Index,Grocery,Housing,Utilities,Transportation,Health,Misc.,mean_price,pizza_index_NY,diff
15,22,Kentucky,KY,81.529210,65.489528,54.142680,74.314855,73.754517,55.333398,74.735109,13.380000,92.867104,11.337894
4,34,Colorado,CO,89.862543,73.573027,92.487394,70.330564,78.513923,74.113437,78.668326,10.559167,73.288433,-16.574110
9,46,Hawaii,HI,101.804124,145.009654,172.384818,147.633471,94.894737,102.678730,112.211934,12.712500,88.234160,-13.569964
42,30,Utah,UT,88.316151,76.024728,81.866948,70.030763,81.942821,69.727271,78.073299,15.084706,104.699025,16.382874
46,9,West Virginia,WV,77.319588,65.562228,45.700925,62.705920,73.865448,67.621426,63.303752,10.507397,72.929115,-4.390473


In [67]:
other_df["tuple"] = other_df.apply(lambda x : (x.Index, x.pizza_index_NY), axis=1)

In [68]:
zipped = dict(zip(other_df["State"], other_df["tuple"].apply(tuple)))

In [69]:
def write_json_file(to_add, new_feature_name):
    f = open("data/map-data.json", "r")
    y = json.load(f)
    for elem in y["features"]:
        elem_to_add = elem["properties"]["name"]
        if elem_to_add != "Puerto Rico":
            elem["properties"][new_feature_name] = to_add[elem_to_add]
    f.close()
    f = open("data/map-data.json", "w")
    json.dump(y, f)
    f.close()

In [70]:
write_json_file(zipped, "indexes")

In [10]:
write_json_file(restos_num_by_state, "num_of_restos")
write_json_file(restos_by_pop, "restos_by_pop")
write_json_file(restos_coords, "restos_coords")


In [91]:
new.loc[new.state=="New York"]

,state,count,pop,lat,long,restos_by_pop
32,New York,374,19453561,42.652843,-73.757874,19.225272
